<!-- # Assignment 1 Scientific Computing  -->

# Scientific Computing: Wave and Diffusion Simulations

## Imports

In [ ]:
import numpy as np
from IPython.display import HTML

import src.solutions as solutions
import src.visualizations as visualizations


In [ ]:
colors = ["orange", "blue", "green", "purple", "brown"]

## Waves

The discretized wave equation becomes (with functions from solutions): 

### Initial values for discretized wave function
Euler's method: f_x+1 = f_x + h * f'_x

(c*delta t)/delta x < 1 for stable computation 
looking at this method, you can derive that in the first two time-points, the function follows the exact same curve as the derivative (f'_x) is 0

### B: Plotted snapshots of the wave at different times

In [ ]:
L = 1
N = 100
c = 1
deltat= 0.001
iterations = 30000
overall_solutions=[]

# save a solution for every 3000 timesteps to plot
for i in range(3):
    overall_solution, xs = solutions.one_b_wrapper(i+1, L, N, c, deltat, iterations)
    overall_solutions.append(overall_solution)

In [ ]:
visualizations.visualization_1b(overall_solutions, xs, deltat, iterations)

### C: Wave Animation 

In [ ]:
%matplotlib inline
wave_anim = visualizations.animate_1c(L, N, c, deltat)
saved_anim = "plots/network_animation10.gif"
HTML(f'<img src="{saved_anim}" style="width: 600px;">')

## Time Dependent Diffusion

### D: Boundaries of the domain

The equations for the boundaries:

c^(k+1)_(0, j) = 1 -> top row <br>
c^(k+1)_(N-1, j) = 0 -> bottom row <br>
c^(k+1)_(i, 0) = c^(k+1)_(i, N - 2) -> first column <br>
c^(k+1)_(i, -1) = c^(k+1)_(i, 1) -> last column <br>

In [ ]:
# values used for initial diffusion setup 
# main parameter values (for discretization)
N = 100
L = 1.0
D = 1

# setp sizes
dx = L / N
dt = 0.25 * dx**2

# initial setup
gamma = (D * dt) / (dx**2) #Placeholder that combines diffusion coefficient D, time step dt and spatial step dx
num_steps = int(1.0/ dt)
y_values = np.linspace(0, 1, N)
c = solutions.initialize_grid(N)

par_values = (c, num_steps, N, gamma, dt)

### E: Comparison analytical with numerical solution

Test the correctness of your simulation. Compare to the analytic solution, plot c(y) for different times.

In [ ]:

# parse data and check directory layout and file existence or create new data (set create_new_data = True) 
create_new_data = True
data_file = "2D_diffusion_2.pkl"
comparison = True
all_c, times = solutions.check_and_parse_data(data_file, create_new_data, par_values, comparison)
# visualize the data together with analytical solution. 
if comparison:
    visualizations.plot_analytical_solution_with_error(y_values, all_c, times, D)


### F: Snapshot of 5 Diffusion configurations

Plot the results, show the 2D domain, with a color representing the concentration at each point. Make a plot of the state of the system at several times: t = {0, 0.001, 0.01, 0.1, and 1}

In [ ]:
# t values 0, 0.001, 0.01, 0.1, 1.0
visualizations.plot_five_states(all_c, times)


### G: Animation

Make an animated plot of the time dependent diffusion equation until equilibrium.

In [ ]:


# data parsing/generation 
data_file = "2D_diffusion_2.pkl"
create_new_data = True
comparison = False
all_c, times = solutions.check_and_parse_data(data_file, create_new_data, par_values, comparison)

# visualizations.plot_simulation_without_animation(all_c, N)

# animation 
%matplotlib inline
c2 = c.copy()
anim = visualizations.animate_1g(solutions.update, c2, num_steps, N, gamma, dt)
HTML(anim.to_jshtml())

## Steady State Configuration 

In [ ]:
# parameters for questions H t/m J: Iterative Solvers 

max_iters = 10000 # maximum number of iterations
N = 50 # grid size 

# analytical solution
tol = 1e-6 # tolerance for stopping criterion
omega = 1.9 # omega value for SOR

# convergence measure vs iterations
p_values = np.arange(0, 11) # p values for stopping criterion
omegas = [1.7, 1.8, 1.9] # omega values for SOR

# optimal omega vs grid size
omega_range = np.arange(1.7, 2.0, 0.05) # omega values
N_values = [10, 20, 50, 100] # grid sizes
optimal_omegas = {}
iters_N = {}

### H: Jacobi, Gauss-Seidel, SOR VS. Analytical

 Implemented the Jacobi, Gauss-Seidel, and Successive over Relaxation methods. Compared the results to the analytical result for N=50.

In [ ]:
# get final grids for all three iterative methods
_, c_jacobi_2d = solutions.sequential_jacobi(N, tol, max_iters)
_, c_gs_2d     = solutions.sequential_gauss_seidel(N, tol, max_iters)
_, c_sor_2d    = solutions.sequential_SOR(N, tol, max_iters, omega)

visualizations.visualization_1h(c_jacobi_2d, c_gs_2d, c_sor_2d, N)

### I: Convergence for different tol δ vs Number of Iterations 

Shows how the convergence measure δ depends on the number of iterations k for each of the methods.

In [ ]:
# get iteration count for all three iterative methods
results_jacobi = [solutions.sequential_jacobi(N=N, tol=10.0**-p, max_iters=max_iters) for p in p_values]
iterations_jacobi, _ = zip(*results_jacobi)

results_gauss_seidel = [solutions.sequential_gauss_seidel(N=N, tol=10.0**-p, max_iters=max_iters) for p in p_values]
iterations_gauss_seidel, _ = zip(*results_gauss_seidel)

results_sor = {omega: [solutions.sequential_SOR(N=N, tol=10.0**-p, max_iters=max_iters, omega=omega) for p in p_values] for omega in omegas}
iterations_sor = {omega: [result[0] for result in results_sor[omega]] for omega in omegas}

visualizations.visualization_1i(p_values, iterations_jacobi, iterations_gauss_seidel, iterations_sor, colors=colors)

### J: Finding optimal ω for SOR

In the SOR method, finds the optimal ω and how it depends on N.

In [ ]:
# loop over grid sizes
for N in N_values:
    best_omega = None
    min_iters = float('inf')

    iters_omega = []

    # loop over omegas 
    for omega in omega_range:

        # get iteration count for SOR method
        iters, _ = solutions.sequential_SOR(N=N, tol=1e-6, max_iters=max_iters, omega=omega)

        iters_omega.append(iters)

        # if current omega results in fewer iterations than current minimum iteration count
        if iters < min_iters:
            
            # update min_iters and best_omega for this grid size
            min_iters = iters
            best_omega = omega

    # store best omega for this grid size in lost of optimal omegas
    optimal_omegas.append(best_omega)
    
    # store iterion counts for all omega values for this grid size
    iters_N[N] = iters_omega

visualizations.visualization_1j_omega_iters(iters_N, omega_range, colors=colors)
visualizations.visualization_1j_N_omegas(N_values, optimal_omegas, colors=colors)

### K: Objects

#### Define Parameters and Create Object Grids

In [ ]:
# parameters for experimentation 
max_iters = 10000 #after this we stop trying to converge

omega_range = np.arange(1.7, 2.0, 0.05)
omega_range = np.round(omega_range, 2)
omega_range_c = np.delete(omega_range, -1) #omegas for experimentation 

N_values = [20, 50, 100] #experimental vlaues for grid size
omegatje= 1.9 #optimal omega for grid size 50X50
tol = 1e-6 #default tol
PROCESSES = 10  #number of processes for parallelization 
N=50 #default grid size
num_grids = 10 # number of object grids with the same configuration are created (as they're placed randomly multiple runs are needed)


In [ ]:

# different object configurations (note that the covered surface for the first 3 configs is the same)
object_configs = [
    (3, 8),   # 3 objects of size 8x8
    (48, 2),  # 48 objects of size 2x2
    (12, 4),  # 12 objects of size 4x4
    (6, 4)   # 6 objects of size 4x4
]


# naming of the object configurations run with SOR
sizes = [
        "3 of 8×8",
        "48 of 2×2",
        "12 of 4×4",
        "6 of 4×4", 
        "0 of 0x0"
    ]

# Saving all grids in a dictionary. 
all_grids = dict()
for ntje in N_values:
    # skip if N is smaller than 20, than the objects are too big in contrast to the gridsizes
    if ntje <20:
        continue

    # objects (following configuration) are randomly placed on the grid
    object_grids = solutions.create_object_layouts(ntje, object_configs, num_grids)
    all_grids[ntje] = object_grids
    # take an examplatory grid to visualize

# Visualization of 50x50 grid with all object configurations 
visualizations.visualize_object_grid(all_grids[50], sizes)


#### Apply SOR on different grids for all object configuraitons (parallelized over different runs)

In [ ]:
# get the mean, variance of every grid size for every object configuration (takes 1 min)
# null metric is for grid without any objects 
all_results, solutions_map, null_metric = solutions.generate_grid_results(N_values, N, all_grids, num_grids, max_iters, omegatje, tol, object_configs, "N", PROCESSES)

#### Apply SOR on different omega values for all object configuraitons (parallelized over different runs)


In [ ]:
# get the mean, variance of every omega for every object configuration (takes 1 min)
# null metric is for grid without any objects
all_results_omega, solutions_map_omega, null_metric_omega = solutions.generate_grid_results(omega_range_c, N, all_grids, num_grids, max_iters, omegatje, tol, object_configs, "O", PROCESSES)

#### Visualize results for different grid sizes and omega values for all object configurations

In [ ]:
visualizations.vis_object_per_gridsize(all_results, all_results_omega, null_metric, null_metric_omega, object_configs, sizes, colors)

#### Visualizing the converged grids for every object configuration setting and omega values (one examplatory object layout is used for each parameter setting)
this to aid the understanding of why convergence is reached faster when more objects are placed 

In [ ]:
for config_label in object_configs:
    visualizations.plot_converged_object_grid(solutions_map_omega, omega_range_c, config_label, "O")

#### Visualizing the converged grids for every object configuration setting and all grid sizes (one examplatory object layout is used for each parameter setting)

In [ ]:

for i, config_label in enumerate(object_configs):
    visualizations.plot_converged_object_grid(solutions_map, N_values, config_label)



#### Statistical Testing for every object configuration
As we used multiple runs because of random placement of the objects, statistical testing is necessary

In [ ]:
solutions.statistical_test_for_objects(object_configs, all_results_omega)
solutions.statistical_test_for_objects(object_configs, all_results, "N")